## AlphaNeo for Max Length of 1 (depth of 2)

```
tensorboard --logdir runs
```

```
cd ./Trinity/
python ./AlphaNeo_Cambrian_pworker.py 1
```

```
nohup jupyter lab > jupyter.log &
```

In [1]:
DBG_VAR = None
DBG_SEXP = None
DBG_POST = None

In [2]:
import logging 
logging.basicConfig(level=logging.CRITICAL)

In [3]:
import os
import itertools
import copy
import random
import fcntl

from pathlib import Path

In [4]:
import tyrell.spec as S
from tyrell.interpreter import Interpreter, PostOrderInterpreter, GeneralError, InterpreterError
from tyrell.enumerator import Enumerator, SmtEnumerator, RandomEnumerator, DesignatedEnumerator, RandomEnumeratorS, ExhaustiveEnumerator
from tyrell.decider import Example, ExampleConstraintPruningDecider, ExampleDecider, TestDecider
from tyrell.synthesizer import Synthesizer
from tyrell.logger import get_logger
from sexpdata import Symbol
from tyrell import dsl as D
from typing import Callable, NamedTuple, List, Any

In [5]:
# import pickle
import dill as pickle
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from tensorboardX import SummaryWriter

use_cuda = torch.cuda.is_available()
print("use_cuda: {}".format(use_cuda))

use_cuda: True


In [6]:
# Morpheus Version
from utils_morpheus import *
from ProgramSpace import *

In [7]:
torch.__version__

'1.0.0'

In [8]:
class ListModule(object):
    def __init__(self, module, prefix, *args):
        self.module = module
        self.prefix = prefix
        self.num_module = 0
        for new_module in args:
            self.append(new_module)
    
    def append(self, new_module):
        if not isinstance(new_module, nn.Module):
            raise ValueError('Not a Module')
        else:
            self.module.add_module(self.prefix + str(self.num_module), new_module)
            self.num_module += 1
            
    def __len__(self):
        return self.num_module
    
    def __getitem__(self, i):
        if i<0 or i>=self.num_module:
            raise IndexError('Out of bound')
        return getattr(self.module, self.prefix+str(i))

In [9]:
class AlphaNeo(nn.Module):
    def __init__(self, p_config=None):
        super(AlphaNeo, self).__init__()
        self.config = p_config
        
        # abstraction token embedding
        self.vocab_size = self.config["embd_vocab_size"]
        self.embd_dim = self.config["embd_dim"]
        self.embedding = nn.Embedding(
            self.vocab_size,
            self.embd_dim,
            self.config['IDX_PAD'],
        )
        
        self.convs = ListModule(self, "convs_")
        self.pools = ListModule(self, "pools_")
        for i in range(len(self.config["conv_n_kernels"])):
            self.convs.append(
                nn.Conv2d(
                    in_channels = self.config["embd_dim"],
                    out_channels = self.config["conv_n_kernels"][i],
                    kernel_size = self.config["conv_kernel_sizes"][i],
                )
            )
            self.pools.append(
                nn.MaxPool2d(
                    kernel_size = self.config["pool_kernel_sizes"][i],
                    padding = self.config["IDX_PAD"],
                )
            )
        
        self.lstm = nn.LSTMCell(
            input_size = self.config["lstm_input_size"],
            hidden_size = self.config["lstm_hidden_size"],
            bias=True,
        )
        
        self.fcs = ListModule(self, "fc_")
        self.fcs.append(
            nn.Linear(self.config["lstm_hidden_size"], self.config["fcs"][0])
        )
        for i in range(1,len(self.config["fcs"])):
            self.fcs.append(
                nn.Linear(self.config["fcs"][i-1], self.config["fcs"][i])
            )
    
    '''
    single batch behavior, no batch dim expected
    '''
    def forward(self, pmaps, hc=None):
        # pmaps: list of maps
        # hc: previous hidden state, if None, start a new sequence
        
        d_embds = [
            self.embedding(pmaps[i]).permute(0,3,1,2)
            for i in range(self.config["n_maps"])
            # (1, nrow, ncol) -> (1, nrow, ncol, dim) -> (1, dim, nrow, ncol), fit the conv
        ]
        d_convs = [
            F.relu(self.convs[i](d_embds[i])) 
            for i in range(self.config["n_maps"])
            # (1, dim ,nrow, ncol) -> (1, n_kernel, nrow, 1)
        ]
        d_pools = [
            self.pools[i](d_convs[i]).view(1,self.config["conv_n_kernels"][i]) 
            for i in range(self.config["n_maps"])
            # (1, n_kernel, nrow, 1) -> (1, n_kernel, 1, 1) -> (1, n_kernel)
        ]
        
        d_known = torch.cat(d_pools,dim=1) # (1, n_kernel*n_maps)
        
#         print("====SHAPE INFOR====")
#         print("embd:{}".format(d_embd.shape))
#         print("conv:{}".format(d_conv.shape))
#         print("pool:{}".format(d_pool.shape))
#         print("known:{}".format(d_known.shape))
        
        if hc is None:
            hc = self.init_hidden()
        
        h_t, c_t = self.lstm(d_known, hc)
        
        tmp1 = h_t
        for i in range(len(self.fcs)-1):
            tmp1 = F.relu(self.fcs[i](tmp1))
        
        # (1, #production_rules)
        tmp1 = F.log_softmax(self.fcs[len(self.fcs)-1](tmp1), dim=1)
        
        # hidden states can be reused
        return tmp1, (h_t,c_t)
    
    def init_hidden(self):
        if use_cuda:
            return (torch.zeros(1,self.config["lstm_hidden_size"]).cuda(),
                    torch.zeros(1,self.config["lstm_hidden_size"]).cuda())
        else:
            return (torch.zeros(1,self.config["lstm_hidden_size"]),
                    torch.zeros(1,self.config["lstm_hidden_size"]))

In [10]:
def AlphaNeoTrainer(p_config, p_spec, p_interpreter, p_generator, p_model, p_optim, p_writer):
    global DBG_VAR, DBG_SEXP, DBG_POST
    reward_list = []
    n_batch = 1
    batch_loss = 0.
    c_nth = 0
    
    # #### mp #### #
    for ii in [1,2,3]:
        data_path = "./pworker_storage/data_{}.pkl".format(ii)
        Path(data_path).touch()
    external_data = []
    # iterator
    iter_external_data = iter(external_data)
    need_reload = True
    # #### mp #### #
    
    total_ac = {1:0,2:0,3:0}
    total_sp = {1:0,2:0,3:0}
    # one program each step
    for d_step in range(p_config["n_steps"]):
        p_model.train()
        
        # #### mp #### #
        if need_reload:
            for ii in [1,2,3]:
                data_path = "./pworker_storage/data_{}.pkl".format(ii)
                # compete once and get some results
                f = open(data_path,"rb+")
                try:
                    fcntl.flock(f.fileno(), fcntl.LOCK_EX)
                    if os.path.getsize(data_path)>0:
                        external_data = pickle.load(f)
                        if len(external_data)>0:
                            # actually it has data
                            f.seek(0)
                            pickle.dump([],f)
                            iter_external_data = iter(external_data)
                            need_reload = False
                except IOError as e:
                    if e.errno != errno.EAGAIN:
                        # unrelated errors
                        raise
                    else:
                        # fail once, move on generate by itself
                        pass
                f.close()
                if not need_reload:
                    break
        # #### mp #### #
        
        try:
            p_prog, str_example = next(iter_external_data)
            # Morpheus ONLY: convert str_example to p_example
            # assign new names
            p_example = Example(
                input=[get_fresh_name() for p in str_example.input],
                output=get_fresh_name(),
            )
            # load the value into new names in R environment
            for i in range(len(p_example.input)):
                p_interpreter.load_data_into_var(
                    str_example.input[i], # data
                    p_example.input[i], # var name
                )
            p_interpreter.load_data_into_var(
                str_example.output,
                p_example.output,
            )
        except StopIteration:
            need_reload = True
            # run out of sample, generate one 
            while True:
                p_input = p_interpreter.random_table()
                p_prog, p_example = p_generator.generate(
                    fixed_depth=p_config["max_depth"],
                    example=Example(input=[p_input], output=None),
                )
                # make sure at least one function call
                if p_prog is not None and p_prog.is_apply():
                    break
                
        # construct the full program
        ps_full = ProgramSpaceChainOneNB(
            p_spec, p_interpreter, eq_r, p_example.input, p_example.output,
        )
        p_prog_list = ps_full.get_prog_list(p_prog)
        for p in p_prog_list:
            ps_full.add_sexp(p.to_sexp())
        total_sp[len(ps_full.prog_list)] += 1
        # start from the first state
        ps_current = ProgramSpaceChainOneNB(
            p_spec, p_interpreter, eq_r, p_example.input, p_example.output,
        )
        d_reward = None
        selected_edges = []
        
        # if true, the problem turns into supervised problem
        if random.random()<=p_config["hint_rate"](d_step,len(ps_full.prog_list)):
            need_hints = True
        else:
            need_hints = False
            

        # roll till the ending condition
        hc_t = None
        while True:
            outv_current = ps_current.get_frontier()
            
            d_maps = camb_get_features(outv_current[0], ps_current.output)
            # a tuple ()

            if use_cuda:
                td_maps = [
                    Variable(torch.LongTensor( [d_maps[i]] )).cuda()
                    for i in range(len(d_maps))
                ]
            else:
                td_maps = [
                    Variable(torch.LongTensor( [d_maps[i]] ))
                    for i in range(len(d_maps))
                ]
                                     
            # (1, LIST_PAD_LENGTH+#production_rules)
            td_output, hc_t = p_model(td_maps, hc_t)
            cnd_list = ps_full.shell_list

            DBG_VAR = [ps_full, ps_current]
            if need_hints:
                # just assign the correct solution
                selected_id = ps_full.str_shell_dict[
                    str(ps_full.prog_list[len(ps_current.prog_list)])
                ]
            else:
                if random.random()<=p_config["exploration_rate"](d_step):
                    # exploration
                    selected_id = random.choice(range(len(cnd_list)))
                else:
                    # exploitation
                    selected_id = torch.multinomial(td_output.exp().flatten(), 1).cpu().flatten().numpy()[0]
            # keep track of selected edges
            selected_edges.append(td_output[0, selected_id])
            # add selected edges and fill
            ret = ps_current.add_sexp(ps_current.shell_list[selected_id].to_sexp())
            # debug info
            DBG_SEXP = ps_current.shell_list[selected_id].to_sexp()
            DBG_POST = ps_current
            
            if ret==False:
                # failed
                d_reward = -0.1
                hc_t = None # cut off sequence
                break
            else:
                # #### disabled temporarily #### #
                # #### should be used together with sanity_check, but now the outv from generator may not pass outv_check #### #
                # outv check!! can terminate early
                # outv_post = ps_current.get_frontier()
                # if not p_interpreter.outv_check(outv_post[0]):
                    # fail the check
                    # d_reward = 0.
                    # hc_t = None
                    # break
                # succeeded
                if ps_current.out_eq():
                    # solved in depth less than or equal to max_depth
                    d_reward = 1.
                    total_ac[len(ps_current.prog_list)] += 1
                    hc_t = None # cut off sequence
                    break
                # NOTICE: Trinity depth is # of layers of nodes, AlphaNeo depth is # of edges (height)
                elif len(ps_current.prog_list)>=p_config["max_depth"]-1:
                    d_reward = -0.1
                    hc_t = None # cut off sequence
                    break
                    
        
        # finally compute the loss
        d_loss = 0.
        ns = len(selected_edges)
        for i in range(ns):
            d_decay = p_config["gamma"]**(ns-1-i)
            # should negate the log probabilities
            d_loss += d_decay*d_reward*(-selected_edges[i])

        reward_list.append(d_reward)
        batch_loss += d_loss

        print("\r# STEP{}, from:{}, size:c{}/f{}, reward:{:.4f}, avg.reward:{:.4f}, ac/sp: 1->{}/{}, 2->{}/{}, 3->{}/{}".format(
            d_step, "gen" if need_reload else "load",
            len(ps_current.prog_list), len(ps_full.prog_list), d_reward, sum(reward_list)/len(reward_list),
            total_ac[1], total_sp[1], total_ac[2], total_sp[2], total_ac[3], total_sp[3],
        ), end="")

        if writer is not None:
            writer.add_scalar(
                'avg.reward/step',
                sum(reward_list)/len(reward_list),
                d_step,
            )
        
        
        c_nth += 1
        if c_nth%n_batch==0:
            c_nth = 0
            # perform gradient in every batch
            batch_loss.backward()
            p_optim.step()
            p_optim.zero_grad()
            batch_loss = 0.
            
#         if d_step%10000==0:
#             torch.save(p_model.state_dict(), "./saved_models/0610_RL1_AlphaNeo_mChainOneNB_ep{}.pt".format(d_step))


In [11]:
m_interpreter = MorpheusInterpreter()
m_spec = S.parse_file('./example/set_unite.tyrell')
m_eq = eq_r
m_generator = MorpheusGenerator(
    spec=m_spec,
    interpreter=m_interpreter,
    sfn=m_interpreter.sanity_check,
)
m_ps = ProgramSpaceChainOneNB(
    m_spec, m_interpreter, m_eq, None, None,
)
m_config = {
    'n_maps': 4, # for amap, bmap, cmap, dmap
    'embd_vocab_size': len(CAMB_LIST),
    'embd_dim': 10,
    'IDX_PAD': 0, 
    'n_steps': 10000000,
    'gamma': 0.618,
    'exploration_rate': lambda x:0.9-0.8*(min(1, x/2500)),
    'hint_rate': lambda x,n:{1:0.8}[n]*(1-(min(0.8, x/2500))),
    'max_depth': 2,
}
m_config["conv_n_kernels"] = [10 for _ in range(m_config["n_maps"])]
m_config["conv_kernel_sizes"] = [
    (1,CAMB_NCOL) 
    for _ in range(m_config["n_maps"])
]
m_config["pool_kernel_sizes"] = [
    (1,1), (CAMB_NROW,1), (1,1), (CAMB_NROW,1)
    # a:1, b:nrow, c:1, d:nrow
] 
m_config["lstm_input_size"] = sum(m_config["conv_n_kernels"])
m_config["lstm_hidden_size"] = 256
m_config["fcs"] = [
    len(m_ps.shell_list),
]

alpha_neo = AlphaNeo(p_config=m_config)
if use_cuda:
    alpha_neo = alpha_neo.cuda()
optimizer = torch.optim.Adam(list(alpha_neo.parameters()))
writer = SummaryWriter("runs/0625CAMB_RL1LSTM_unite")
# writer = None

In [12]:
m_config

{'n_maps': 4,
 'embd_vocab_size': 574,
 'embd_dim': 10,
 'IDX_PAD': 0,
 'n_steps': 10000000,
 'gamma': 0.618,
 'exploration_rate': <function __main__.<lambda>(x)>,
 'hint_rate': <function __main__.<lambda>(x, n)>,
 'max_depth': 2,
 'conv_n_kernels': [10, 10, 10, 10],
 'conv_kernel_sizes': [(1, 20), (1, 20), (1, 20), (1, 20)],
 'pool_kernel_sizes': [(1, 1), (50, 1), (1, 1), (50, 1)],
 'lstm_input_size': 40,
 'lstm_hidden_size': 256,
 'fcs': [36]}

In [13]:
len(m_ps.shell_list)

36

In [14]:
AlphaNeoTrainer(m_config, m_spec, m_interpreter, m_generator, alpha_neo, optimizer, writer)

# STEP5857, from:gen, size:c1/f1, reward:1.0000, avg.reward:0.7824, ac/sp: 1->4699/5858, 2->0/0, 3->0/0

KeyboardInterrupt: 